<h3>Determining Inference Using MultiNLI - Unmatched Dataset</h3>
<b>Term Paper</b>

<b>Chris Huber\
CS820\
Prof. Kulkari, Spring 2022\
</b> 

In [30]:
import pandas as pd
import numpy as np

import torch

In [31]:
# Download RoBERTa already finetuned for MNLI
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

with torch.no_grad():
    # Encode a pair of sentences and make a prediction
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 0  # contradiction

    # Encode another pair of sentences
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 2  # entailment

Using cache found in /Users/chrishuber/.cache/torch/hub/pytorch_fairseq_main
2022-03-24 06:39:56 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz from cache at /Users/chrishuber/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
2022-03-24 06:39:57 | INFO | fairseq.tasks.masked_lm | dictionary: 50264 types
2022-03-24 06:40:09 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 8, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_

In [32]:
roberta.register_classification_head('new_task', num_classes=3)
logprobs = roberta.predict('new_task', tokens)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
print(logprobs)
print(logprobs.argmax(dim=1))

tensor([[-0.9398, -1.0406, -1.3624]], grad_fn=<LogSoftmaxBackward0>)
tensor([0])


In [33]:
roberta.register_classification_head('unmatched_mlni', num_classes=3)

In [34]:
test_df = pd.read_json("./input/multinli_0.9_test_mismatched_unlabeled.jsonl", lines=True)
test_df.head(5)

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[hidden, hidden, hidden, hidden, hidden]",oup,hidden,0,0,"Even here, the channel perspective tells a som...","( ( Even here ) ( , ( ( the ( channel perspect...","(ROOT (S (ADVP (RB Even) (RB here)) (, ,) (NP ...",The story told by the channel is slightly diff...,( ( ( The story ) ( told ( by ( the channel ) ...,(ROOT (S (NP (NP (DT The) (NN story)) (VP (VBN...
1,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,1,1,My theory is that the r got put in there simp...,( ( My theory ) ( ( is ( that ( ( the r ) ( ( ...,(ROOT (S (NP (PRP$ My) (NN theory)) (VP (VBZ i...,I think the r is there just because people are...,( I ( ( think ( ( the r ) ( ( is there ) ( jus...,(ROOT (S (NP (PRP I)) (VP (VBP think) (SBAR (S...
2,"[hidden, hidden, hidden, hidden, hidden]",verbatim,hidden,2,2,To go on using in its former sense a word whos...,( ( To ( go ( on ( ( using ( in ( its ( former...,(ROOT (S (S (VP (TO To) (VP (VB go) (PP (IN on...,To go on using it in its former sense could be...,( ( To ( go ( on ( ( using it ) ( in ( its ( f...,(ROOT (NP (S (S (VP (TO To) (VP (VB go) (PP (I...
3,"[hidden, hidden, hidden, hidden, hidden]",nineeleven,hidden,3,3,"He alerted his supervisor, who assigned anothe...","( He ( ( alerted ( ( ( his supervisor ) , ) ( ...",(ROOT (S (NP (PRP He)) (VP (VBD alerted) (NP (...,The supervisor had another controller work wit...,( ( The supervisor ) ( ( had ( ( another ( con...,(ROOT (S (NP (DT The) (NN supervisor)) (VP (VB...
4,"[hidden, hidden, hidden, hidden, hidden]",letters,hidden,4,4,I have also enclosed a brief report on those a...,( I ( ( ( have also ) ( ( enclosed ( a ( brief...,(ROOT (S (NP (PRP I)) (VP (VBP have) (ADVP (RB...,Your annual contribution will have little impact.,( ( Your ( annual contribution ) ) ( ( will ( ...,(ROOT (S (NP (PRP$ Your) (JJ annual) (NN contr...


In [35]:
print(test_df.shape[0])

9847


In [36]:
# Encode a pair of sentences and make a prediction
tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
roberta.predict('mnli', tokens).argmax()  # 0: contradiction

tensor(0)

In [37]:
sentences1 = list(test_df['sentence1'])
sentences2 = list(test_df['sentence2'])
test_pair_ids = list(test_df['pairID'])

In [38]:
test_s1 = test_df['sentence1']
test_s2 = test_df['sentence2']
print(test_s1[0])
print(test_s2[0])

Even here, the channel perspective tells a somewhat different story.
The story told by the channel is slightly different.


In [39]:
from fairseq.data.data_utils import collate_tokens

batch_of_pairs = [
    ['Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.'],
    ['Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.'],
    ['potatoes are awesome.', 'I like to run.'],
    ['Mars is very far from earth.', 'Mars is very close.'],
]

batch = collate_tokens(
    [roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
)

logprobs = roberta.predict('mnli', batch)
print(logprobs.argmax(dim=1))

tensor([0, 2, 1, 0])


In [40]:
%%time

# initialize counters
count_c = 0
count_e = 0
count_n = 0

# This uses the pretrained Roberta MLNI model to make predictions about what class the two sentences are and assigns the output to a list
test_preds = []
with torch.no_grad():
    for k in range(len(test_s1)):
        # Encode a pair of sentences and make a prediction
        print("Pair " + str(k) + ":")
        print(test_s1[k], test_s2[k])
        tokens = roberta.encode(test_s1[k], test_s2[k])
        prediction = roberta.predict('mnli', tokens).argmax().item()
        
        # make sublist from pairIDs and semantic prediction
        test_pred = []
        test_pred.append(test_pair_ids[k])
        
        # 0 is a contradiction, 1 is neutral, and 2 is an entailment
        if prediction == 0:
            test_pred.append("contradiction")
            count_c += 1
        elif prediction == 2:
            test_pred.append("entailment")
            count_e += 1
        else:
            test_pred.append("neutral")
            count_n += 1
        test_preds.append(test_pred)
        print(prediction)
        print("-----------------")

print("done!")

Pair 0:
Even here, the channel perspective tells a somewhat different story. The story told by the channel is slightly different.
2
-----------------
Pair 1:
My theory is that the  r got put in there simply because muskrat is a more common word than muscat . It is the same reason most of us I think the r is there just because people aren't smart.
0
-----------------
Pair 2:
To go on using in its former sense a word whose meaning has changed is counterproductive. To go on using it in its former sense could be considered more practical
0
-----------------
Pair 3:
He alerted his supervisor, who assigned another controller to assist him. The supervisor had another controller work with him.
2
-----------------
Pair 4:
I have also enclosed a brief report on those areas which would be helped by your annual contribution. Your annual contribution will have little impact.
0
-----------------
Pair 5:
// A wise decision// yeah. I approve of the decision because it's what I would have done. 
1
----

2
-----------------
Pair 44:
When, um, I was little, I lived more, near Lake Norman side of Charlotte. I really loved the time I spent on the Lake Norman side.
1
-----------------
Pair 45:
Indeed, joint participation in make-believe with more expert cultural members is so important for development during early childhood that I devote an entire chapter to it. Make-believing with more experienced cultural members positively impacts a child in early childhood development.
2
-----------------
Pair 46:
This landlord is quite a character. The landlord was dull and typical. 
0
-----------------
Pair 47:
Seconds later she said,Oh my God we are way too low. She knew they were too low.
2
-----------------
Pair 48:
The NEADS commander told us he did not pass along the order because he was unaware of its ramifications. The commander told people to follow the order.
0
-----------------
Pair 49:
And I said, Why did you come back, Victoria? I asked, "Why did you come back, Victoria?"
2
--------------

2
-----------------
Pair 83:
Authors/Publishers/ Editors/Agents Thrown to the  Reviews and Reviewers Authors often make useful reviewers and critics.
1
-----------------
Pair 84:
Uh, you don't have to answer to anybody except yourself. I'm the boss, you must answer to me.
0
-----------------
Pair 85:
And the day that you put the, the shower and the tub in. I remember the day you put the shower and the tub in, it was a really hot and humid day, honestly.
1
-----------------
Pair 86:
 Nothing in life is all mazel [`good luck']. There is just good luck for Jewish people.
1
-----------------
Pair 87:
So American 11 isn't the hijack at all then, right? American 11 crash was due other reasons than hijacking.
1
-----------------
Pair 88:
It was kind of like eating it with whipping cream. It was like eating something mixed with whipping cream.
2
-----------------
Pair 89:
Do you like the weather in Charlotte? What's your opinion on Charlotte's weather?
2
-----------------
Pair 90:
And you don'

2
-----------------
Pair 129:
As news of the hijackings filtered through the FAA and the airlines, it does not seem to have occurred to their leadership that they needed to alert other aircraft in the air that they too might be at risk. The FAA immediately told all planes about the threat.
0
-----------------
Pair 130:
It's just that, what a firefight! The firefight was postponed.
0
-----------------
Pair 131:
The growing presence of fashion-basic elements in myriad consumer products means that all retailers and suppliers may find new competitive opportunities using replenishment. Fashion-basic elements are continually dwindling from presence in consumer products.
0
-----------------
Pair 132:
The fact that the pictures are a full page each with captions that seesaw across the bottoms (as palindromes will) would be frustrating were it not for the quality of the drawings. The pictures are tiny.
0
-----------------
Pair 133:
Among these   Get Back  (a sign flashed on TV to invite or comm

0
-----------------
Pair 166:
Then her father introduces a special form of strategic thinking called distancing. Strategic thinking consists of multiple methods, however distancing is not considered one of them.
0
-----------------
Pair 167:
Just a little longer, and then we'll go inside.  We will go in right now.
0
-----------------
Pair 168:
I have sent a check for $100 and hope that you will consider a like gift or more. I would like for you to consider making a donation in the amount similar to mine. 
2
-----------------
Pair 169:
The size of retail establishments also grew during this period because of two important trends. The establishments decreased in size at the time.
0
-----------------
Pair 170:
Contributions will be used to cover program and membership costs for girls in financial need; subsidize the cost of Girl Scout camp; and provide training and resources to leaders, parents and adult volunteers. Most donations are spent for summer camp costs.
1
-----------------
Pair 

...

2
-----------------
Pair 9671:
The roots of these polarized perspectives can be found in centuries-old, dramatically opposing philosophies about the nature of children and child development. Recent advances in science has not relieved many of these conflicts.
1
-----------------
Pair 9672:
Basically if she couldn't hit a note he would digitize it so that she did. He would digitize notes she couldn't hit. 
2
-----------------
Pair 9673:
At farmers' markets, families would display the vegetables they grew or sell eggs from their chickens. The families would sell produce that others had grown. 
2
-----------------
Pair 9674:
Or high school? Or University?
0
-----------------
Pair 9675:
There would probably be little chance of a refund if the place were run by a feminist. Feminists always give refunds, no matter what the situation is.
0
-----------------
Pair 9676:
Everyone at the IU School of Dentistry is excited about the upcoming Fall Dental Alumni Conference to be held in Bloomington o

2
-----------------
Pair 9708:
Yiddish is especially versatile in describing those poor souls who inhabit the world of the ineffectual, and each is assigned a distinct place in the gallery of pathetic  schmo, schmendrik, schnook, schmegegge, schlep, schlub, schmuck, putz, klutz, kvetch , and nudnik .  Yiddish easily coins new names for new  a nudnik is a `pest'; a phudnik is a `nudnik with a Ph.D.'  The rich nuances that suffuse this roll call are seen in the timeless distinction between a schlemiel `clumsy jerk' and a schlimazel `habitual loser': the schlemiel inevitably trips and spills his hot soup'all over the schlimazel. Speakers of Yiddish are often focused on the personality trait of competence.
0
-----------------
Pair 9709:
   Discussion with my gun-loving friends supports Funk's penultimate etymology. This etymology is supported by talking with friends who are gun enthusiasts.
2
-----------------
Pair 9710:
An integral part of that reputation has been the professional success

1
-----------------
Pair 9745:
Well, I remember another book that you read that that I didn't read but that, uh, painting was made `84? I really have a long list of books to read that you recommended me.
1
-----------------
Pair 9746:
According to Rousseau, adult training served only to thwart the child's inherently perceptive intelligence and moral sense, which unfolded naturally as children moved through a sequence of developmental stages. Adult training thwarted child smartness and sense of morality, Rousseau wrote.
2
-----------------
Pair 9747:
One of the areas we've decided to focus on is development. We want to improve our development
1
-----------------
Pair 9748:
If I had, if I was retired, I would definitely be working in aliteracy, um, you know, volunteer literacy capacity. If I had, I would definitely be working with children.
2
-----------------
Pair 9749:
When the local civil aviation security office of the Federal Aviation Administration (FAA) later investigated these se

2
-----------------
Pair 9783:
The big crate. The shipping container.
2
-----------------
Pair 9784:
Piaget's contribution to the field of child development is enormous. Piaget was the village idiot, who didn't know a thing about raising children.
0
-----------------
Pair 9785:
In this jeremiad, males are viewed as the  enemy,  and are so characterized throughout the book, which concludes with exhortations to engage the foe and a strident call to arms (though not men's). This book is not about male vs female struggles.
0
-----------------
Pair 9786:
I can't imagine not reading. I can not picture myself not reading.
2
-----------------
Pair 9787:
In Johnson's poem this becomes a  fasting Mounseer.  This is a fasting Mounseer in Johnson's poem.
2
-----------------
Pair 9788:
Plans to attack the United States were developed with unwavering singlemindedness throughout the 1990s. There were no plans to attack the United States in the 1990s.
0
-----------------
Pair 9789:
Right after the Pen

1
-----------------
Pair 9822:
What would change for that boy or girl? What new circumstance would that boy or girl find themselves in?
2
-----------------
Pair 9823:
65 percent of participants will indicate that being in YMCA programs has helped them stay out of trouble. YMCA program participants have lower incidents of crime then non-participants. 
1
-----------------
Pair 9824:
'  It is not clear why Heinlein chose such an unpretty word for such a beautiful concept (maybe it is euphonious to the Martian auditory apparatus); it has, nevertheless, caught on. Heinlein used an unattractive word to describe an appealing concept.
2
-----------------
Pair 9825:
That won't bother you? Are you sure that's okay with you?
2
-----------------
Pair 9826:
The behaviorist presumption that development can be mechanically engineered by social input, guaranteeing brighter, socially more mature children, is not borne out by the evidence. It has been proven that development can be mechanically engineer

In [42]:
# print prediction counts as a sanity check
print("Contradictions: " + str(count_c))
print("Entailments: " + str(count_e))
print("Neutrals: " + str(count_n))

Contradictions: 3265
Entailments: 3561
Neutrals: 3021


In [43]:
# turn nested lists into a dataframe
submission_df = pd.DataFrame(test_preds, columns = ['pairID', 'gold_label'])
submission_df.head

<bound method NDFrame.head of       pairID     gold_label
0          0     entailment
1          1  contradiction
2          2  contradiction
3          3     entailment
4          4  contradiction
...      ...            ...
9842    9842     entailment
9843    9843        neutral
9844    9844        neutral
9845    9845        neutral
9846    9846     entailment

[9847 rows x 2 columns]>

In [44]:
# export probabilities for words in each class to .csv file
submission_df.to_csv("mismatched_submission.csv", index=False)

![title](kaggle_mismatched_score.png)